In [5]:
import cubewalkers as cw
import pystablemotifs as psm
import pyboolnet as pbn

In [8]:
cacc_rules = '''
SMAD,	!JUN & TGFR
CYTC,	MOMP
TNFA,	MAC
FAS,	CTL
GP130,	IL6
CASP8,	!P21 & !CFLIP & FADD
SPHK1,	!(!TNFR & !ERK)
JAK,	GP130 & !SOCS
SOCS,	STAT3
SMASE,	!(!P53 & !FADD)
TH1,	!IL10 & IL12 & !IL4 & !TGFB | IFNG & !IL10 & !IL4 & !TGFB
IFNG,	!(!TH1 & !CTL)
SOD,	!(!STAT3 & !NFKB)
PTEN,	!JUN & P53 & !NFKB
IL10,	!(!TREG & !TH2)
JUN,	BCATENIN & JNK & !GSK3B | JNK & !GSK3B & ERK
PP2A,	CERAMIDE & !AKT
P53,	PTEN & !MDM2 | JNK & !MDM2 | ATM & !MDM2
RAS,	!(!GP130 & !EP2)
CASP3,	CASP8 & !IAP | !IAP & CASP9
MEK,	!(!RAF & !ROS)
MAC,	IFNG & !IL10 | !IL10 & CCL2
P21,	SMAD & !CASP3 & !GSK3B | P53 & !CASP3 & !GSK3B
CERAMIDE,	!SPHK1 & SMASE
TREG,	DC & !IL6 | IL10 & !IL6
TBID,	CASP8 & !BCL2
IL12,	!(!MAC & !DC)
BCATENIN,	!(GSK3B & APC)
RAF,	!(!RAS & !CERAMIDE)
ASK1,	!P21 & ROS
TNFR,	TNFA
TH2,	!IFNG & IL4 & !TGFB
IL4,	!(!TH2 & !DC)
STAT3,	JAK
JNK,	!(!ASK1 & !MEKK1)
CFLIP,	NFKB
IKK,	!(!TNFR & !AKT | !S1P & !AKT)
EP2,	PGE2
SMAD7,	!(!SMAD & !NFKB)
PI3K,	!PTEN & RAS | !PTEN & EP2
CTL,	IFNG & !TGFB
FOS,	ERK
BCL2,	!PP2A & !P53 & NFKB | !PP2A & !P53 & STAT3
TGFR,	!SMAD7 & TGFB
GSK3B,	!EP2 & !AKT
CCL2,	NFKB
MOMP,	TBID & !BCL2 | CERAMIDE & !BCL2 | !BCL2 & BAX
ATM,	ROS
IAP,	STAT3 & !SMAC | !SMAC & NFKB
Apoptosis,	CASP3
Proliferation,	!CASP3 & !P21 & FOS & CYCLIND1
PGE2,	COX2
COX2,	TNFR & S1P
SMAC,	MOMP
ERK,	MEK
S1P,	SPHK1
ROS,	!SOD & TNFR
NFKB,	!IKB
DC,	TNFA & !IL10 | !IL10 & CCL2
MDM2,	P53 & !GSK3B & !ATM & AKT
FADD,	!(!FAS & !TNFR)
TGFB,	TREG
CYCLIND1,	JUN & !GSK3B | BCATENIN & !GSK3B | STAT3 & !GSK3B
IKB,	!IKK
IL6,	!(!MAC & !NFKB & !DC)
CASP9,	CYTC & !P21 & !IAP
MEKK1,	!(!CERAMIDE & !TNFR & !TGFR)
BAX,	PP2A & P53 & !AKT | PP2A & TBID & !AKT
AKT,	!PP2A & !CASP3 & PI3K
APC,	APC
'''

In [9]:
cacc_primes = psm.format.create_primes(cacc_rules)

fixed_off_names = {'DC', 'IL12', 'IL4', 'TH1', 'TH2', 'TREG', 'IFNG', 'MAC', 'IL10', 'CCL2', 'IL6', 'TFGB', 'CTL', 'TNFA', 'PGE2'}
fixed = {n:0 for n in fixed_off_names}
fixed['APC']=1
perc,_=psm.drivers.logical_domain_of_influence(fixed,cacc_primes)
pbn.prime_implicants.percolate(cacc_primes,perc,remove_constants=True,copy=False)
psm.format.pretty_print_prime_rules(cacc_primes)

Apoptosis* = CASP3
BAX* = P53 & PP2A
CASP3* = CASP9
CASP9* = CYTC
CERAMIDE* = SMASE & !SPHK1
CYTC* = MOMP
ERK* = MEK
FOS* = ERK
JNK* = MEKK1
MEK* = RAF
MEKK1* = CERAMIDE
MOMP* = CERAMIDE | BAX
P53* = PTEN | JNK
PP2A* = CERAMIDE
PTEN* = P53
RAF* = CERAMIDE
S1P* = SPHK1
SMAC* = MOMP
SMASE* = P53
SPHK1* = ERK


In [16]:
cacc_model=cw.Model(psm.format.primes2bnet(cacc_primes))
cacc_model_original=cw.Model(cacc_rules)

In [18]:
SFQC = cacc_model.quasicoherence(n_time_steps=10000, n_walkers=5000, T_sample=1000,
                          fuzzy_coherence=True, maskfunction=cw.update_schemes.synchronous)
N = cacc_model.n_variables
print(f'{N*(1-SFQC)=}')

N*(1-SFQC)=array(0.348139)


In [20]:
SFQC_O = cacc_model_original.quasicoherence(n_time_steps=10000, n_walkers=5000, T_sample=1000,
                          fuzzy_coherence=True, maskfunction=cw.update_schemes.synchronous,threads_per_block=(16,16))
N_O = cacc_model_original.n_variables
print(f'{N_O*(1-SFQC_O)=}')

N_O*(1-SFQC_O)=array(1.12033524)


In [22]:
hgsd_rules = '''
WNT4,	!DKK1 & RSPO1 & !FGF9 | !DKK1 & CTNNB1 & !FGF9 | !DKK1 & GATA4 & !FGF9 | !DKK1 & NR0B1 & !FGF9
PGD2,	SOX9
DHH,	SOX9
DMRT1,	!FOXL2 & SRY | !FOXL2 & SOX9
WT1pKTS,	!(!UGR & !GATA4 | WNT4 & CTNNB1)
DKK1,	!(!SRY & !SOX9)
NR0B1,	WNT4 & CTNNB1 & !NR5A1 | WNT4 & CTNNB1 & !SOX9 | WT1mKTS & !SOX9 | WT1mKTS & !NR5A1
UGR,	!WNT4 & UGR & !NR5A1
GATA4,	!(!WNT4 & !UGR & !SRY & !NR5A1)
CBX2,	!WNT4 & UGR | !NR0B1 & UGR | UGR & !CTNNB1
RSPO1,	WNT4 & !DKK1 | !DKK1 & CTNNB1
FOXL2,	WNT4 & !DMRT1 & CTNNB1 & !SOX9
AMH,	!NR0B1 & GATA4 & NR5A1 & SOX9 | GATA4 & !CTNNB1 & NR5A1 & SOX9
CTNNB1,	WNT4 & !AMH & !SRY | WNT4 & !SRY & !SOX9 | RSPO1 & !AMH & !SRY | RSPO1 & !SRY & !SOX9
SRY,	!CTNNB1 & SOX9 | !CTNNB1 & SRY | WT1pKTS & GATA4 & CBX2 & !CTNNB1 & NR5A1 | CBX2 & !CTNNB1 & WT1mKTS & NR5A1
WT1mKTS,	!(!UGR & !GATA4)
NR5A1,	!(!UGR & !GATA4 & !CBX2 & !WT1mKTS | WNT4 & NR0B1)
SOX9,	!WNT4 & !FOXL2 & !CTNNB1 & SOX9 & FGF9 | !WNT4 & !FOXL2 & !CTNNB1 & SRY | !WNT4 & PGD2 & !FOXL2 & !CTNNB1
FGF9,	!WNT4 & SOX9
'''

In [31]:
hgsd_primes = psm.format.create_primes(hgsd_rules)

fixed_off_names = {'UGR'}
fixed = {n:0 for n in fixed_off_names} # change the `n:0` to `n:1` to see effect of UGR=1
perc,_=psm.drivers.logical_domain_of_influence(fixed,hgsd_primes)
pbn.prime_implicants.percolate(hgsd_primes,perc,remove_constants=True,copy=False)
psm.format.pretty_print_prime_rules(hgsd_primes)

AMH* = GATA4 & !NR0B1 & NR5A1 & SOX9 | !CTNNB1 & GATA4 & NR5A1 & SOX9
CTNNB1* = !SOX9 & !SRY & WNT4 | RSPO1 & !SOX9 & !SRY | !AMH & !SRY & WNT4 | !AMH & RSPO1 & !SRY
DHH* = SOX9
DKK1* = SRY | SOX9
DMRT1* = !FOXL2 & SRY | !FOXL2 & SOX9
FGF9* = SOX9 & !WNT4
FOXL2* = CTNNB1 & !DMRT1 & !SOX9 & WNT4
GATA4* = WNT4 | SRY | NR5A1
NR0B1* = CTNNB1 & !SOX9 & WNT4 | CTNNB1 & !NR5A1 & WNT4 | !SOX9 & WT1mKTS | !NR5A1 & WT1mKTS
NR5A1* = !WNT4 & WT1mKTS | !NR0B1 & WT1mKTS | GATA4 & !WNT4 | GATA4 & !NR0B1
PGD2* = SOX9
RSPO1* = !DKK1 & WNT4 | CTNNB1 & !DKK1
SOX9* = !CTNNB1 & FGF9 & !FOXL2 & SOX9 & !WNT4 | !CTNNB1 & !FOXL2 & SRY & !WNT4 | !CTNNB1 & !FOXL2 & PGD2 & !WNT4
SRY* = !CTNNB1 & SRY | !CTNNB1 & SOX9
WNT4* = !DKK1 & !FGF9 & RSPO1 | !DKK1 & !FGF9 & NR0B1 | !DKK1 & !FGF9 & GATA4 | CTNNB1 & !DKK1 & !FGF9
WT1mKTS* = GATA4
WT1pKTS* = GATA4 & !WNT4 | !CTNNB1 & GATA4


In [32]:
hgsd_model=cw.Model(psm.format.primes2bnet(hgsd_primes))
hgsd_model_original=cw.Model(hgsd_rules)

In [33]:
SFQC = hgsd_model.quasicoherence(n_time_steps=10000, n_walkers=5000, T_sample=1000,
                          fuzzy_coherence=True, maskfunction=cw.update_schemes.synchronous)
N = hgsd_model.n_variables
print(f'{N*(1-SFQC)=}')

N*(1-SFQC)=array(1.50432941)


In [27]:
SFQC_O = hgsd_model_original.quasicoherence(n_time_steps=10000, n_walkers=5000, T_sample=1000,
                          fuzzy_coherence=True, maskfunction=cw.update_schemes.synchronous,threads_per_block=(32,32))
N_O = hgsd_model_original.n_variables
print(f'{N_O*(1-SFQC_O)=}')

N_O*(1-SFQC_O)=array(1.4672)
